#### Genero audio con 4 personas, para probar la predicción del modelo.

In [1]:
from keras.models import load_model
from keras.models import model_from_json
import json
from pydub import AudioSegment
import pandas as pd
import numpy as np
from scipy.fftpack import fft
import glob


Using TensorFlow backend.


In [20]:
with open('../models/10_epoch.json','r') as f:
    model_json = json.load(f)

model = model_from_json(model_json)
model.load_weights('../models/10_epoch.h5')


In [21]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 512)               163840512 
_________________________________________________________________
dense_12 (Dense)             (None, 256)               131328    
_________________________________________________________________
batch_normalization_3 (Batch (None, 256)               1024      
_________________________________________________________________
dense_13 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_14 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_15 (Dense)             (None, 10)                1290      
Total params: 164,023,562
Trainable params: 164,023,050
Non-trainable params: 512
______________________________________

In [22]:
people_mix = ['3764_0', '3764_1','8461_0', '8461_1', '8461_2', '6432_0', '8461_3', '8461_4', '2033_0', '2033_1', '6432_1', '6432_2', '6432_3', '3764_2']
len(people_mix)


14

Creo un audio en el que sólo hablan 4 personas de forma intercalada --> "Audio original"

A este audio le aplico la limpieza:

- Split en 20'
- Overlap 50%
- FFT --> features


In [23]:
audio_original = 0
for p in people_mix:
    audio_original += AudioSegment.from_file(f"../outputs/selection/{p}.mp3", format='mp3')


In [24]:
audio_original

In [25]:
len(audio_original)

280000

In [37]:
#Exporto audio_mix para pruebas:

audio_original.export("../outputs/audio_mix.mp3", format="mp3")

<_io.BufferedRandom name='../outputs/audio_mix.mp3'>

In [26]:
# Audios de 20' con overlab 50%:

ftt = []
interval = 20000
counter = 0
num = (len(audio_original)//interval)*2

for i in list(range(num)):
    if counter == 0:
        splited = audio_original[0:interval]
        ftt.append(list(np.abs(fft(splited.get_array_of_samples()))))
        counter+=int(interval/2)
        interval+=int(counter)
    else: 
        splited = audio_original[counter:interval]
        ftt.append(list(np.abs(fft(splited.get_array_of_samples()))))
        counter+=10000
        interval+=10000


print('FIN!')


FIN!


In [27]:
df= pd.DataFrame()

df['ftt'] = ftt[:-1]

In [28]:
df.ftt

0     [5660.0, 6146.923850902667, 3015.403158484773,...
1     [3201.0, 2061.205610321675, 3063.6763366931464...
2     [9897.0, 9032.73268591476, 5220.715290732333, ...
3     [39189.0, 32106.077413535797, 13155.2477826564...
4     [109705.0, 67920.98584428975, 45095.8354531856...
5     [166304.0, 15436.88483738735, 126984.827258543...
6     [108982.0, 232540.8085622957, 207278.110050192...
7     [39126.0, 155757.48325105628, 217308.821777671...
8     [209968.0, 96090.17798635119, 150409.787742716...
9     [188344.0, 49993.23752061838, 70564.7479165986...
10    [112376.0, 97876.9258603135, 66299.75061536973...
11    [616579.0, 587856.0284589797, 664614.284038967...
12    [460244.0, 710047.9307859545, 642972.782997177...
13    [42939.0, 176160.11349335773, 182876.594994446...
14    [25881.0, 128391.70301034994, 142322.811115458...
15    [1192185.0, 1219869.9340780328, 801421.5976190...
16    [428317.0, 1012657.6309960033, 2015952.8319187...
17    [1492875.0, 164080.67088658616, 2810249.00

In [29]:
X = np.vstack(df.ftt)
X.shape

(27, 320000)

In [30]:
predictions = model.predict(X)

In [31]:
df['y_predict'] = [list(e) for e in predictions]
df['label_predict'] = df['y_predict'].apply(lambda x: x.index(max(x)))

In [32]:
predictions[9]

array([2.9300744e-04, 2.6262459e-10, 1.3591360e-09, 5.2567022e-03,
       3.4259342e-02, 3.3848971e-04, 4.4877506e-08, 9.5984668e-01,
       2.8139508e-11, 5.6875183e-06], dtype=float32)

In [33]:
df

,ftt,y_predict,label_predict
0,"[5660.0, 6146.923850902667, 3015.403158484773,...","[0.019940024, 5.4029217e-05, 0.001436354, 0.35...",5
1,"[3201.0, 2061.205610321675, 3063.6763366931464...","[0.024312718, 0.00031330533, 0.010538717, 0.34...",3
2,"[9897.0, 9032.73268591476, 5220.715290732333, ...","[0.012077111, 2.2348004e-06, 0.00016922217, 0....",4
3,"[39189.0, 32106.077413535797, 13155.2477826564...","[0.00080534886, 6.926819e-10, 4.967265e-07, 0....",4
4,"[109705.0, 67920.98584428975, 45095.8354531856...","[2.0620126e-05, 2.7859036e-13, 9.1622443e-10, ...",4
5,"[166304.0, 15436.88483738735, 126984.827258543...","[0.00024734385, 8.163798e-10, 1.2535044e-07, 0...",4
6,"[108982.0, 232540.8085622957, 207278.110050192...","[2.8467408e-08, 1.0165669e-20, 6.9880525e-16, ...",4
7,"[39126.0, 155757.48325105628, 217308.821777671...","[2.190099e-08, 2.8409344e-21, 3.2734483e-16, 2...",4
8,"[209968.0, 96090.17798635119, 150409.787742716...","[8.596699e-06, 9.363705e-15, 1.1249464e-10, 1....",4
9,"[188344.0, 49993.23752061838, 70564.7479165986...","[0.00029300744, 2.626246e-10, 1.359136e-09, 0....",7


In [34]:
speakers_predict = df.label_predict.to_list()

In [35]:
speakers = []
memory = 0

for i, e in enumerate (speakers_predict):
    if i == 0:
        speakers.append(e)
    elif i< len(speakers_predict)-1 and speakers_predict[i]!= speakers_predict[i+1] and speakers_predict[i]!= speakers[-1]:
        speakers.append(e)
    elif i == len(speakers_predict)-1 and speakers_predict[i]!= speakers[-1]:
        speakers.append(e)

        
speakers        

[5, 3, 4, 7, 4, 7, 6, 7, 5]

In [36]:
df.label_predict.value_counts()

7    12
4    11
5     2
6     1
3     1
Name: label_predict, dtype: int64